# <b> LIBRARIES

In [1]:
import pandas as pd
import numpy as np
!pip3 install yfinance
import yfinance as yf
import datetime as dt
!pip3 install pandas_datareader
from pandas_datareader import data as pdr
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import os
from pandas import ExcelWriter

In [2]:
yf.pdr_override()

# <B> Back-Testing

In [4]:
#Input
stock     = input("Enter a stock ticker symbol : ")
startyear = input("Enter a starting year       : ")
startmonth= input("Enter a starting month      : ")
startday  = input("Enter a starting date       : ")



#Date
start = dt.datetime(int(startyear), int(startmonth), int(startday))
now = dt.datetime.now()
df = pdr.get_data_yahoo(stock,start,now)



#Exponential Moving Average
emasUsed = [3,5,8,10,12,15,30,35,40,45,50,60]
for ema in emasUsed:
    df["ema_" + str(ema)]=round(df.iloc[:,4].ewm(span=ema, adjust=False).mean(),2)
    
df=df.iloc[60:]

pos=0 #position
num=0 #binaries (1 = position, 0 = no position)
percentchange=[] 

for i in df.index:
    cmin = min(df["ema_3"][i],df["ema_5"][i],df["ema_8"][i],df["ema_10"][i],df["ema_12"][i],df["ema_15"][i])
    cmax = max(df["ema_30"][i],df["ema_35"][i],df["ema_40"][i],df["ema_45"][i],df["ema_50"][i],df["ema_60"][i])
    
    close = df["Adj Close"][i]
    
    if(cmin>cmax): #Buying Point
        # print("Red White Blue")
        if(pos==0):
            bp=close #buying point = Adj Close
            pos=1
            # print("Buying now at "+str(bp))
            
    elif(cmin<cmax): #Selling Point
        # print("Blue White Red")
        if(pos==1):
            pos=0
            sp=close
            # print("Selling now at "+str(sp))
            pc=(sp/bp-1)*100
            percentchange.append(pc)
            
    if(num==df["Adj Close"].count()-1 and pos==1):
        pos=0
        sp=close
        # print("Selling now at "+str(sp))
        pc=(sp/bp-1)*100
        percentchange.append(pc)
    num+=1

print(percentchange)
    
gains=0 
ng=0 #number of gains
losses=0
nl=0 #number of losses
totalR=1 #Total return

for i in percentchange:
    if (i>0):
        gains+=i
        ng+=1
    else:
        losses+=i
        nl+=1
    totalR=totalR*((i/100)+1)
    
totalR=round((totalR-1)*100,2)

if(ng>0):
    avgGain=gains/ng
    maxR=str(max(percentchange))
else:
    avgGain=0
    maxR="undefined"
    
if(nl>0):
    avgLoss=losses/ng
    maxL=str(min(percentchange))
    ratio=str(-avgGain/avgLoss)
else:
    avgLoss=0
    maxL="undefined"
    ratio="inf"
    
if(ng>0 or nl>0): #Success Rate
    battingAvg=ng/(ng+nl)
else:
    battingAvg=0
    
print()
print("Results for "+ stock + " going back to "+str(df.index[0]) +", Sample size: "+str(ng+nl)+" trades")
print("EMAs used: "+str(emasUsed))
print("Batting Avg: "+ str(battingAvg))
print("Gain/loss ratio: "+ ratio)
print("Average Gain: "+ str(avgGain))
print("Average Loss: "+ str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ " trades: "+ str(totalR)+"%" )
# print("Example return Simulating "+str(n)+ " trades: "+ str(nReturn)+"%" )
print()

Enter a stock ticker symbol :  AAPL
Enter a starting year       :  2021
Enter a starting month      :  01
Enter a starting date       :  01


[*********************100%***********************]  1 of 1 completed
[-4.894740790397201, 12.559313122359029, 14.363983162731797, -2.200595294269847, -2.262855810663489, 0.0]

Results for AAPL going back to 2021-03-30 00:00:00, Sample size: 6 trades
EMAs used: [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]
Batting Avg: 0.3333333333333333
Gain/loss ratio: 2.876976299078112
Average Gain: 13.461648142545414
Average Loss: -4.679095947665268
Max Return: 14.363983162731797
Max Loss: -4.894740790397201
Total return over 6 trades: 17.02%

